# Data Center Energy Analysis - Zip 93309

This notebook analyzes energy usage patterns for ZIP code 93309, utilizing other 933xx ZIP codes as a baseline for comparison. The goal is to identify major usage spikes potentially indicative of data center loads. It includes a specific check for "Load Masking" where high demand might be shifted to neighboring ZIP codes.

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

warnings.filterwarnings('ignore')

# Load the cleaned dataset
data_path = 'pge_electric_usage_2015_2025_cleaned.csv'
combined_df = pd.read_csv(data_path)

print(f"✓ Data loaded: {combined_df.shape}")

## Filter Data
Filtering for Commercial, Residential, and Industrial customers.

In [ ]:
valid_customer_classes = ['Elec- Commercial', 'Elec- Residential', 'Elec- Industrial']
combined_df = combined_df[combined_df['CUSTOMERCLASS'].isin(valid_customer_classes)].copy()

print(f"✓ Filtered to relevant classes: {len(combined_df):,} rows")

## Analysis Setup for 93309

In [ ]:
# Define ZIP codes
data_center_zip = 93309
comparison_zips = [93301, 93304, 93305, 93306, 93307, 93308, 93311, 93312, 93313, 93314]
all_target_zips = [data_center_zip] + comparison_zips

print("Analysis Setup:")
print(f"  Target ZIP: {data_center_zip}")
print(f"  Comparison ZIPs: {comparison_zips}")

# Filter for target ZIPs
target_data = combined_df[combined_df['ZIPCODE'].isin(all_target_zips)].copy()

# Create year-month column
target_data['YEAR_MONTH'] = pd.to_datetime(
    target_data['YEAR'].astype(str) + '-' + target_data['MONTH'].astype(str).str.zfill(2) + '-01'
)

print(f"\n✓ Filtered to target ZIPs: {len(target_data):,} rows")

## Load Masking Detection & Correction
Checking for instances where usage might be masked by shifting it to neighboring ZIPs.

In [ ]:
def detect_and_correct_load_masking(data, data_center_zip, neighbor_zips):
    """
    Detects load masking in data center ZIP and reallocates usage from neighbors.
    """
    corrected_data = data.copy()
    reallocation_log = []
    
    # Calculate baseline (median) for each ZIP
    baselines = data.groupby('ZIPCODE')['TOTALKWH'].median().to_dict()
    dc_baseline = baselines.get(data_center_zip, 0)
    
    # Define suspicious threshold (50% below baseline)
    suspicious_threshold = dc_baseline * 0.5
    
    # Get data center monthly records
    dc_records = corrected_data[corrected_data['ZIPCODE'] == data_center_zip].copy()
    
    print(f"Load Masking Detection for ZIP {data_center_zip}")
    print("=" * 80)
    print(f"Baseline (median) usage: {dc_baseline:,.0f} kWh")
    print(f"Suspicious threshold (50% of baseline): {suspicious_threshold:,.0f} kWh")
    
    suspicious_count = 0
    reallocated_count = 0
    
    for idx, row in dc_records.iterrows():
        if row['TOTALKWH'] < suspicious_threshold:
            suspicious_count += 1
            year_month = row['YEAR_MONTH']
            original_usage = row['TOTALKWH']
            
            # Check neighboring ZIPs for excess usage during this period
            neighbor_data = corrected_data[
                (corrected_data['ZIPCODE'].isin(neighbor_zips)) &
                (corrected_data['YEAR_MONTH'] == year_month)
            ]
            
            # Find ZIP with highest excess above its baseline
            best_candidate = None
            max_excess_pct = 0
            
            for _, neighbor_row in neighbor_data.iterrows():
                neighbor_zip = neighbor_row['ZIPCODE']
                neighbor_usage = neighbor_row['TOTALKWH']
                neighbor_baseline = baselines.get(neighbor_zip, neighbor_usage)
                
                if neighbor_baseline > 0:
                    excess = neighbor_usage - neighbor_baseline
                    excess_pct = (excess / neighbor_baseline) * 100
                    
                    # Consider candidates with 50%+ excess above baseline
                    if excess_pct > 50 and excess_pct > max_excess_pct:
                        max_excess_pct = excess_pct
                        best_candidate = {
                            'zip': neighbor_zip,
                            'usage': neighbor_usage,
                            'baseline': neighbor_baseline,
                            'excess': excess,
                            'excess_pct': excess_pct,
                            'idx': neighbor_row.name
                        }
            
            # If strong candidate found, reallocate the excess
            if best_candidate and best_candidate['excess'] > 0:
                reallocated_count += 1
                
                # Amount to reallocate (the excess usage)
                reallocation_amount = best_candidate['excess']
                
                # Update data center usage
                corrected_data.loc[idx, 'TOTALKWH'] = original_usage + reallocation_amount
                
                # Reduce neighbor's reported usage
                neighbor_idx = best_candidate['idx']
                corrected_data.loc[neighbor_idx, 'TOTALKWH'] = best_candidate['usage'] - reallocation_amount
                
                # Log the reallocation
                reallocation_log.append({
                    'date': year_month,
                    'dc_original': original_usage,
                    'dc_corrected': original_usage + reallocation_amount,
                    'source_zip': best_candidate['zip'],
                    'source_original': best_candidate['usage'],
                    'source_corrected': best_candidate['usage'] - reallocation_amount,
                    'reallocated_kwh': reallocation_amount,
                    'excess_pct': best_candidate['excess_pct']
                })
    
    print(f"\nSuspicious low-usage periods detected: {suspicious_count}")
    print(f"Reallocations performed: {reallocated_count}")
    
    if reallocated_count > 0:
        print(f"\nSample Reallocations (first 5):")
        print("-" * 80)
        for i, log in enumerate(reallocation_log[:5]):
            date_str = log['date'].strftime('%b %Y')
            print(f"{date_str}: Reallocated {log['reallocated_kwh']:,.0f} kWh from ZIP {log['source_zip']}")
            print(f"  Target: {log['dc_original']:,.0f} → {log['dc_corrected']:,.0f} kWh")
            print(f"  Source ({log['source_zip']}): {log['source_original']:,.0f} → {log['source_corrected']:,.0f} kWh")

    return corrected_data, reallocation_log

# Pre-aggregate monthly data for all ZIPs
monthly_detection = target_data.groupby(['ZIPCODE', 'YEAR_MONTH']).agg({
    'TOTALKWH': 'sum',
    'TOTALCUSTOMERS': 'sum'
}).reset_index()

# Apply load masking correction
corrected_monthly, reallocation_log = detect_and_correct_load_masking(
    monthly_detection, 
    data_center_zip, 
    comparison_zips # Using local 933xx neighbors as candidates
)

## Aggregate Monthly Usage (Corrected)

In [ ]:
# Split into target and comparison using CORRECTED data
dc_usage = corrected_monthly[corrected_monthly['ZIPCODE'] == data_center_zip].copy()
comp_usage = corrected_monthly[corrected_monthly['ZIPCODE'].isin(comparison_zips)].copy()

# Calculate average of comparison ZIPs
comp_avg = comp_usage.groupby('YEAR_MONTH').agg({
    'TOTALKWH': 'mean',
    'TOTALCUSTOMERS': 'mean'
}).reset_index()

dc_usage_sorted = dc_usage.sort_values('YEAR_MONTH').copy()

print(f"✓ Monthly aggregation complete (using corrected data)")
print(f"  Target Zip Months: {len(dc_usage)}")
print(f"  Comparison Months: {len(comp_avg)}")

## Calculate Statistics

In [ ]:
# Month-over-month changes
dc_usage_sorted['PREV_KWH'] = dc_usage_sorted['TOTALKWH'].shift(1)
dc_usage_sorted['MOM_CHANGE'] = dc_usage_sorted['TOTALKWH'] - dc_usage_sorted['PREV_KWH']
dc_usage_sorted['MOM_PCT_CHANGE'] = (dc_usage_sorted['MOM_CHANGE'] / dc_usage_sorted['PREV_KWH'] * 100)

# Statistics
dc_mean = dc_usage_sorted['TOTALKWH'].mean()
comp_mean = comp_avg['TOTALKWH'].mean()
difference = dc_mean - comp_mean
pct_difference = (difference / comp_mean) * 100

print("Overall Statistics:")
print(f"  Target Zip Average: {dc_mean:,.0f} kWh")
print(f"  Comparison Average: {comp_mean:,.0f} kWh")
print(f"  Difference: {difference:+,.0f} kWh ({pct_difference:+.1f}%)")
print(f"\n  Target Peak: {dc_usage_sorted['TOTALKWH'].max():,.0f} kWh")
print(f"  Comparison Peak: {comp_avg['TOTALKWH'].max():,.0f} kWh")

## Identify Major Usage Spikes

In [ ]:
print("Top 10 Peak Usage Months for 93309:")
print("=" * 70)
top_months = dc_usage_sorted.nlargest(10, 'TOTALKWH')[['YEAR_MONTH', 'TOTALKWH']]
for idx, row in top_months.iterrows():
    date_str = row['YEAR_MONTH'].strftime('%B %Y')
    print(f"{date_str:20} | {row['TOTALKWH']:>12,.0f} kWh")

print("\n\nTop 10 Largest Month-over-Month Increases:")
print("=" * 70)
top_increases = dc_usage_sorted.nlargest(10, 'MOM_CHANGE')[['YEAR_MONTH', 'TOTALKWH', 'MOM_CHANGE', 'MOM_PCT_CHANGE']]
for idx, row in top_increases.iterrows():
    date_str = row['YEAR_MONTH'].strftime('%B %Y')
    print(f"{date_str:20} | Usage: {row['TOTALKWH']:>12,.0f} kWh | Change: +{row['MOM_CHANGE']:>12,.0f} kWh ({row['MOM_PCT_CHANGE']:>6.1f}%)")

## Visualizing the Data with Key Dates
A comparison chart of the target zip vs average of others, annotated with key industry events to provide context.

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(dc_usage_sorted['YEAR_MONTH'], dc_usage_sorted['TOTALKWH'], label='Zip 93309 (Corrected)', linewidth=2.5, color='#1f77b4')
plt.plot(comp_avg['YEAR_MONTH'], comp_avg['TOTALKWH'], label='Avg Comparison Zips (933xx)', linewidth=2, color='gray', linestyle='--', alpha=0.7)

# Add key dates context
key_dates = [
    ('2022-11-30', 'ChatGPT Launch', 'green'),
    ('2023-03-14', 'GPT-4 Release', 'red'),
    ('2024-02-15', 'Sora Announcement', 'purple')
]

for date_str, label, color in key_dates:
    date_val = pd.to_datetime(date_str)
    plt.axvline(x=date_val, color=color, linestyle=':', alpha=0.8)
    # Get y-value at this date for annotation placement (approximate)
    y_max = dc_usage_sorted['TOTALKWH'].max()
    plt.text(date_val, y_max * 0.95, f' {label}', rotation=90, verticalalignment='top', color=color, fontweight='bold')

plt.title('Energy Usage Analysis: Zip 93309 vs Local Baseline', fontsize=16, pad=20)
plt.ylabel('Total kWh', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.legend(fontsize=12, loc='upper left')
plt.grid(True, alpha=0.3)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

# Format x-axis
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()